# Demonstration of the SCF Voting Mechanism for PoC Purposes

*Danilo Lessa Bernardineli (BlockScience), July 2023*

On this Notebook, we'll provide an stylized demonstration for how
Neural Governance could be implemented along with four of its modules: Quorum Voting Neuron, Trust Neuron, Voting History Neuron and Expertise Neuron. Detailed instructions can be found on the [SCF Voting Mechanism PoC Specification](https://hackmd.io/GMs8iB1MQEGsJirqqPm3NA) and [SDF Voting Mechanism PoC Design](https://hackmd.io/HzRrf1NtQ_a7nlSvX_stXg?both) documents.


As a testing benchmark, we'll also generate fictious datasets that
represents the Voting Users, their actions, their trust graph, and the
inputs for the Voting History / Expertise Neuron.

## Code Dependences

In [1]:
from enum import Enum, auto
from functools import reduce
from collections import defaultdict
from dataclasses import dataclass

import networkx as nx

## Definitions

In [2]:
UserUUID = str
ProjectUUID = str
VotingPower = float

## Data Structures

In [3]:
class Vote(Enum):
    Yes = 1.0
    No = -1.0
    Abstain = 0.0

class Action(Enum):
    RoundVote = auto()
    Delegate = auto()
    Abstain = auto()

## User Data Generation

In [4]:
USERS = ['maria', 'fernando', 'giuseppe', 'sarah', 'tom', 'laura']

PROJECTS = {'voting-mech-for-scf', 
            'quorum-voting'}

USER_ACTIONS = {
    'maria':(Action.RoundVote, {'voting-mech-for-scf': Vote.Yes, 
                           'quorum-voting': Vote.Abstain}),
    'fernando': (Action.RoundVote, {'voting-mech-for-scf': Vote.Yes}),
    'giuseppe': (Action.RoundVote, {'voting-mech-for-scf': Vote.Yes, 
                               'quorum-voting': Vote.Yes}),
    'sarah': (Action.RoundVote, {'voting-mech-for-scf': Vote.Yes, 
                            'quorum-voting': Vote.No}),
    'tom': (Action.RoundVote, USERS),
    'laura': (Action.Delegate, ['maria', 'fernando', 'sarah', 'tom'])
}


TRUST_GRAPH = {'maria': ['fernando', 'giuseppe'], 'fernando': ['giuseppe'], 'giuseppe': ['tom']}


## Computing Voting Results

### Neural Governance Definition

In [5]:
# Attribute Voting Power to an (user, project) tuple
def user_project_vote_power(uid: UserUUID, 
                            pid: ProjectUUID, 
                            neuron_layers: tuple[dict, callable],
                            initial_votes: float=0.0) -> VotingPower:
    """
    Computes an User vote towards an Project as based on 
    an Feedforward implementation of Neural Governance for an strictly
    sequential network (no layer parallelism).
    """
    current_vote = initial_votes
    for layer in neuron_layers:
        (neurons, layer_aggregator) = layer
        neuron_votes = []
        for (neuron_label, neuron) in neurons.items():
            (oracle_function, weighting_function) = neuron
            raw_neuron_vote = oracle_function(uid, pid, current_vote)
            neuron_votes.append(weighting_function(raw_neuron_vote))
        current_vote = layer_aggregator(neuron_votes)
    return current_vote

### Defining Neurons

#### 1st Neuron: Quorum Delegation

In [6]:
def query_user_quorum(user_id: UserUUID,
                      max_quorum_size: int=5) -> list[UserUUID]:
    """
    Retrieves the Actual Quorum for an given delegating user.
    Returns `None` if the User is non-delegating.
    """
    (key, value) = USER_ACTIONS[user_id]
    actual_quorum = None
    if 'key' == 'Delegate':
        actual_quorum = []
        current_delegatees = value
        for delegatee in current_delegatees:
            delegatee_action = USER_ACTIONS[delegatee]

            if delegatee_action == Action.RoundVote:
                actual_quorum.append(delegatee)

            if len(actual_quorum) == max_quorum_size:
                return actual_quorum

        return actual_quorum
    else:
        return None


    
def quorum_participation(quorum: list[UserUUID],
                         project_id: ProjectUUID,
                         quorum_size: int=5) -> float:
    """
    Share of an quorum active participation towards an project.
    """
    active_delegatees = 0
    for delegatee in quorum:
        if USER_ACTIONS[delegatee].get(project_id, None) is not None:
            active_delegatees += 1

    return active_delegatees / quorum_size


def quorum_agreement(quorum: list[UserUUID],
                     project_id: ProjectUUID,
                     initial_agreement: float=0.0) -> float:
    """
    Compute the quorum agreement between active participants
    """
    agreement = initial_agreement
    quorum_size = 0
    for delegatee in quorum:
        action = USER_ACTIONS[delegatee].get(project_id, None)
        # Note: this logic can be different if we weight by User Voting Power
        if action is not None:
            quorum_size += 1
            if action is Vote.Yes:
                agreement += 1
            elif action is Vote.No:
                agreement += -1
            else:
                agreement += 0
    return agreement / quorum_size

def query_user_vote(user_id, project_id) -> Vote | Action:
    
    (action, payload) = USER_ACTIONS.get(user_id, (None, None))
    if action is None:
        return Vote.Abstain
    
    if action is Action.RoundVote:
        project_vote = payload.get(project_id, None)
        if project_vote is None:
            return Vote.Abstain
        else:
            return project_vote

    if action is Action.Delegate:
        return Action.Delegate


def quorum_delegate_result(user_id,
                           project_id,
                           participation_threshold=0.66,
                           agreement_threshold=0.5):
    """
    Oracle Module for the Quorum Delegation Neuron.
    """
    
    vote = query_user_vote(user_id, project_id)

    if vote is Action.Delegate:
        quorum = query_user_quorum(user_id)
        if quorum_participation(quorum, project_id) > participation_threshold:
            agreement = quorum_agreement(quorum, project_id)
            if agreement > agreement_threshold:
                return Vote.Yes
            elif agreement < -agreement_threshold:
                return Vote.No
            else:
                return Vote.Absent
        else:
            return Vote.Absent
    else:
        return vote

#### 2nd Neuron: Reputation Score

In [7]:
class ReputationCategory(Enum):
    Excellent = 4
    VeryGood = 3
    Good = 2
    Average = 1
    Poor = 0
    Uncategorized = -1

@dataclass
class ReputationAPI():
    status: int
    reputation_category: ReputationCategory

    def get(user_id):
        return ReputationAPI(status=200, 
                             reputation_category=ReputationCategory.Uncategorized)


REPUTATION_SCORE_MAP = {
    ReputationCategory.Excellent: 0.3,
    ReputationCategory.VeryGood: 0.2,
    ReputationCategory.Good: 0.1,
    ReputationCategory.Average: 0.05,
    ReputationCategory.Poor: 0.0,
    ReputationCategory.Uncategorized: 0.0
}

def reputation_score(user_id) -> VotingPower:
    """
    Oracle Module for the Reputation Score
    """
    bonus = 0.0
    api_result = ReputationAPI.get(user_id)
    if api_result.status == 200:
        bonus = REPUTATION_SCORE_MAP[api_result.reputation_category]
    return bonus

#### 3rd Neuron: Prior Voting Score

In [8]:
@dataclass
class PastVotingAPI():
    status: int
    active_rounds: list[int]

    def get(user_id):
        return PastVotingAPI(status=200, 
                             active_rounds=[2, 3])

ROUND_BONUS_MAP = {
    1: 0.0,
    2: 0.1,
    3: 0.2,
    4: 0.3
}

def prior_voting_score(user_id, project_id, _) -> VotingPower:
    """
    Oracle Module for the Prior Voting Score
    """

    bonus = 0.0
    api_result = PastVotingAPI.get(user_id)
    if api_result.status == 200:
        user_active_rounds = api_result.active_rounds
        for active_round in user_active_rounds:
            bonus += ROUND_BONUS_MAP[active_round]

#### 4th Neuron: Trust Graph Bonus

In [9]:
# 1) Definitions

# Key is the Trusting User and the Value Set are the Users being Trusted
TrustGraph = dict[UserUUID, list[UserUUID]]

def compute_trust_score(raw_graph: dict) -> dict[UserUUID, float]:
    """
    Computes an Trust Score as based on the Canonical Page Rank.

    This is done by computing the Page Rank on the whole Trust Graph
    with default arguments and scaling the results through MinMax.
    
    The resulting scores will be contained between 0.0 and 1.0
    """
    G = nx.from_dict_of_lists(raw_graph,
                              create_using=nx.DiGraph)

    pagerank_values = nx.pagerank(G, 
                                  alpha=0.85, 
                                  personalization=None, 
                                  max_iter=100,
                                  tol=1e-6,
                                  nstart=None,
                                  weight=None,
                                  dangling=None)
    
    max_value = max(pagerank_values.values())
    min_value = min(pagerank_values.values())
    trust_score = {user: (value - min_value) / (max_value - min_value)
                   for (user, value) in pagerank_values.items()}
    return trust_score

# 2) Backend inputs

TRUST_BONUS_PER_USER = compute_trust_score(TRUST_GRAPH)

# 3) Implementing an Oracle

def trust_score(user_id: UserUUID) -> VotingPower:
    """
    Oracle for the Trust Bonus.
    """
    return TRUST_BONUS_PER_USER[user_id]

### Setting up the Neural Governance Environment

In [11]:

LAYER_1_AGGREGATOR = lambda lst: sum(lst)
# Take the product of the list
LAYER_2_AGGREGATOR = lambda lst: reduce((lambda x, y: x * y), lst) 

LAYER_1_NEURONS = {
    'trust_score': (lambda uid, _1, _2: trust_score(uid),
                    lambda x: x),
    'reputation_score': (lambda uid, _1, _2: reputation_score(uid),
                         lambda x: x)
}

LAYER_2_NEURONS = {
    'power_before_delegation': (lambda _1, _2, prev_vote: prev_vote,
                                lambda x: x),
    'quorum_delegation': (lambda u, p, _: quorum_delegate_result(u, p), 
                          lambda x: x) # Either 0.0 or 1.0
}

NEURAL_GOVERNANCE_LAYERS = [(LAYER_1_NEURONS, LAYER_1_AGGREGATOR),
                            (LAYER_2_NEURONS, LAYER_2_AGGREGATOR)]

# 5) Compute Final Output    

project_vote_power = defaultdict(float)
for project in PROJECTS:
    for user in USERS:
        args = (user, project, NEURAL_GOVERNANCE_LAYERS)
        project_vote_power[project] += user_project_vote_power(*args) 

TypeError: unsupported operand type(s) for *: 'float' and 'Vote'